In [35]:
# make the screen bigger!
from IPython.display import display, HTML
display(HTML(data=""" <style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 85%; }
    div#maintoolbar-container { width: 99%; } </style> """))

import os
import numpy as np
import pandas as pd
import gdal
import ogr
import geopandas as gpd

In [36]:
def createBuffer(inputfn, outputBufferfn, bufferDist):
    inputds = ogr.Open(inputfn)
    inputlyr = inputds.GetLayer()

    shpdriver = ogr.GetDriverByName('ESRI Shapefile')
    if os.path.exists(outputBufferfn):
        shpdriver.DeleteDataSource(outputBufferfn)
    outputBufferds = shpdriver.CreateDataSource(outputBufferfn)
    bufferlyr = outputBufferds.CreateLayer(outputBufferfn, geom_type=ogr.wkbPolygon)
    featureDefn = bufferlyr.GetLayerDefn()

    for feature in inputlyr:
        ingeom = feature.GetGeometryRef()
        geomBuffer = ingeom.Buffer(bufferDist)

        outFeature = ogr.Feature(featureDefn)
        outFeature.SetGeometry(geomBuffer)
        bufferlyr.CreateFeature(outFeature)
        outFeature = None

In [45]:
# set/create GENeral workspace
workspace = os.path.join('.', 'workspace')
if not os.path.exists(workspace):
    os.makedirs(workspace)

# Paths to data
GIS_FOLDER = os.path.join('..', 'Raw_GIS_Data')

# path to the grid bound
Grid_shp = os.path.join(GIS_FOLDER, 'grid_bound.shp')

# path to the raw land use data set
LU_shp = os.path.join(GIS_FOLDER, 'Land_use', 'LU_test_file.shp')
LU_shp_GIANT = os.path.join(GIS_FOLDER, 'Land_use', 'Land_use_WGS84.shp')

if not os.path.exists(os.path.join('.', 'output//')):
    os.makedirs(os.path.join('.', 'output//'))
                        
if not os.path.exists(os.path.join('.', 'output//Figures//')):
    os.makedirs(os.path.join('.', 'output//Figures//'))
fig_path =  (os.path.join('.', 'output//Figures//'))

# define where the land is too steep to build
steep_land = os.path.join(GIS_FOLDER, 'Slopes', 'steep_land_25deg.shp')

In [55]:

# use geopandas to select each landuse independently
dataSrc = gpd.read_file(LU_shp_GIANT)
dataSrc_select = dataSrc[dataSrc['LBLCLASS'] == 'Agroforest']
dataSrc_select.to_file(os.path.join(workspace, 'tempselect2.shp'))

In [58]:
%time createBuffer(os.path.join(workspace, 'tempselect2.shp'), os.path.join(workspace, 'time_the_buff.shp'), 5)

Wall time: 19min 20s


In [22]:

# use geopandas to select each landuse independently
dataSrc = gpd.read_file(LU_shp_GIANT)
dataSrc_select = dataSrc[dataSrc['LBLCLASS'] == 'Agroforest']
dataSrc_select.to_file(os.path.join(workspace, 'tempselect2.shp'))

# use OGR to create the buffer file 
 createBuffer(os.path.join(workspace, 'tempselect2.shp'), os.path.join(workspace, 'bufomomn2.shp'), 1)



In [59]:
type_list =     ['Agroforest', 'Cultivated_Land', 'Developed_Woodlands', 'Open_Space', 'Impervious_Surfaces', 'Buildings']
Buf_dist_list = [5,             4,                 3,                     1,            0.2,                   0.5]

# use geopandas to select each landuse independently
dataSrc = gpd.read_file(LU_shp_GIANT)



for idx, lutype in enumerate(type_list):
    dataSrc_select = dataSrc[dataSrc['LBLCLASS'] == lutype]
    dataSrc_select.to_file(os.path.join(workspace, 'tempselect.shp'))

    # use OGR to create the buffer file 
    print("buffering {}".format(lutype))
    %time  createBuffer(os.path.join(workspace, 'tempselect.shp'), os.path.join(workspace, 'Buffer_{}.shp'.format(lutype)), Buf_dist_list[idx])

buffering Agroforest
Wall time: 23min 10s
buffering Cultivated_Land
Wall time: 44min 50s
buffering Developed_Woodlands
Wall time: 1h 40min 9s
buffering Open_Space


AttributeError: 'NoneType' object has no attribute 'CreateLayer'

buffering Impervious_Surfaces
Wall time: 3h 51min 44s
buffering Buildings


AttributeError: 'NoneType' object has no attribute 'CreateLayer'